# Automated Question-Answer Generation from Pharmaceutical Data

This notebook systematically generates question-answer pairs from pharmaceutical data sources including:
1. PubMed target passages
2. DrugBank tables
3. Related pharmaceutical data

Each QA pair will include:
- Question
- Answer
- Source text/passage
- Related table

In [1]:
# Import required libraries
import os
import pandas as pd
import json
from pathlib import Path
from openai import OpenAI
import csv
from tqdm import tqdm
from collections import defaultdict

In [2]:
# Configuration and paths
PUBMED_TARGETS_DIR = 'inputs/pubmed-targets'
DRUGBANK_TABLES_DIR = 'inputs/drugbank-tables'
MAPPING_FILE = 'inputs/pubmed-drugbank-tables.gt'
OUTPUT_FILE = 'both_output.gt'

# Initialize your LLM API key if needed
# os.environ["OPENAI_API_KEY"]
client = OpenAI()


In [3]:
def load_passage_table_mapping():
    """Load the mapping between passages and their relevant tables"""
    mapping = defaultdict(list)
    with open(MAPPING_FILE, 'r') as f:
        for line in f:
            passage_id, table_name = line.strip().split(',')
            mapping[passage_id].append(table_name)
    # print("mapping", mapping)
    return mapping

def load_target_passages():
    """Load all target passages from the pubmed-targets directory"""
    passages = {}
    target_files = Path(PUBMED_TARGETS_DIR).glob('Target-*')
    
    for file_path in target_files:
        target_id = file_path.name
        with open(file_path, 'r') as f:
            passages[target_id] = f.read()
    # print("passages", passages)
    return passages

def load_drugbank_tables():
    """Load all relevant DrugBank tables"""
    tables = {}
    csv_files = Path(DRUGBANK_TABLES_DIR).glob('*.csv')
    
    for file_path in csv_files:
        table_name = file_path.stem
        tables[table_name] = pd.read_csv(file_path)
    
    # print("tables", tables)
    return tables

In [4]:
def get_relevant_table_content(tables, table_names, max_rows=5):
    """Extract relevant content from tables for context"""
    print("Debug - Available tables:", tables.keys())
    print("Debug - Looking for table_names:", table_names)

    table_content = {}
    for table_name in table_names:
        # Remove .csv extension if present
        base_table_name = table_name.replace('.csv', '')
        
        if base_table_name in tables:
            df = tables[base_table_name]
            table_content[base_table_name] = {
                'columns': list(df.columns),
                'sample': df.head(max_rows).to_dict('records')
            }
        else:
            print(f"Debug - Table '{base_table_name}' not found in available tables")
    return table_content

def generate_questions_for_passage(passage_id, passage_text, tables, relevant_table_names, model="gpt-4o"):
    """Generate questions for a given passage and its relevant tables using LLM"""

    # print("tables", tables)
    # print("relevant_table_names", relevant_table_names)
    
    # Limit passage length if too long (e.g., first 1000 characters)
    passage_text = passage_text[:1000] + "..." if len(passage_text) > 1000 else passage_text
    
    # Limit to maximum 3 relevant tables
    relevant_table_names = relevant_table_names[:3]
    
    # Get relevant table content
    table_content = get_relevant_table_content(tables, relevant_table_names)

    
    prompt = f"""
    Given the following passage and related tables, generate 3-5 meaningful question-answer pairs.
    IMPORTANT: Each question MUST be answerable using information from BOTH the passage AND the tables together.
    Only generate questions that require combining information from both sources.
    Focus on pharmaceutical and medical aspects.
    
    
    Passage (ID: {passage_id}):
    {passage_text}
    
    Related Tables:
    {json.dumps(table_content, indent=2)}
    
    Generate questions in the following format:
    1. question: [specific question about drug/treatment]
       answer: [detailed answer combining information from passage and tables]
       text: [passage ID if information from passage was used (e.g. Target-123456789)]
       table: [table name if information from table was used (e.g. drugbank-drug)]
    
    Ensure every question uses information from both the passage and at least one table.
    """
    
    # Call your LLM here with the prompt
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a medical and pharmaceutical expert tasked with generating detailed question-answer pairs about drugs and treatments."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    print(response.choices[0].message.content)
    
    # Parse the response into structured QA pairs
    qa_pairs = parse_llm_response(response.choices[0].message.content, passage_id)
    return qa_pairs

def parse_llm_response(response_text, passage_id):
    """Parse the LLM response into structured QA pairs"""
    qa_pairs = []
    
    # Split the response into individual QA entries
    entries = response_text.strip().split('\n\n')
    
    for entry in entries:
        if not entry.strip():
            continue
            
        lines = entry.strip().split('\n')
        current_qa = {
            'question': '',
            'answer': '',
            'text': passage_id,
            'table': 'None'  # Default value
        }
        
        for line in lines:
            line = line.strip()
            # Skip empty lines and numbering
            if not line or line.replace('.', '').strip().isdigit():
                continue
                
            # Parse each field using more robust splitting
            if 'question:' in line:
                current_qa['question'] = line.split('question:', 1)[1].strip()
            elif 'answer:' in line:
                current_qa['answer'] = line.split('answer:', 1)[1].strip()
            elif 'text:' in line:
                current_qa['text'] = line.split('text:', 1)[1].strip()
            elif 'table:' in line:
                table_value = line.split('table:', 1)[1].strip()
                # Handle NA, N/A, None cases
                current_qa['table'] = 'None' if table_value.upper() in ['NA', 'N/A', 'NONE'] else table_value
        
        # Only add complete QA pairs that have both question and answer
        if current_qa['question'] and current_qa['answer']:
            qa_pairs.append(current_qa.copy())  # Use copy to avoid reference issues
    
    return qa_pairs

In [9]:
def main():
    # Load mappings and data
    print("Loading passage-table mappings...")
    passage_table_mapping = load_passage_table_mapping()
    
    print("Loading target passages...")
    passages = load_target_passages()
    
    print("Loading DrugBank tables...")
    tables = load_drugbank_tables()
    
    # Initialize output list
    qa_pairs = []
    
    # Process each passage with its relevant tables
    for passage_id, passage_text in tqdm(list(passages.items())[:100]):
        if passage_id in passage_table_mapping:
            relevant_tables = passage_table_mapping[passage_id]
            # print(passage_id, passage_text, tables, relevant_tables)
            
            # Generate QA pairs using the passage and its relevant tables
            new_qa_pairs = generate_questions_for_passage(
                passage_id,
                passage_text,
                tables,
                relevant_tables
            )
            # new_qa_pairs = []
            print(new_qa_pairs)
            qa_pairs.extend(new_qa_pairs)
        else:
            print(f"No table mapping found for passage {passage_id}")
    
    # Save results
    with open(OUTPUT_FILE, 'w', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)
        writer.writerow(['question', 'answer', 'text', 'table'])  # header
        for qa_pair in qa_pairs:
            writer.writerow([
                qa_pair['question'],
                qa_pair['answer'],
                qa_pair['text'],
                qa_pair['table']
            ])
        
    print(f"Generated {len(qa_pairs)} question-answer pairs")

In [10]:
if __name__ == "__main__":
    main()

Loading passage-table mappings...
Loading target passages...
Loading DrugBank tables...


/tmp/ipykernel_214670/2028544267.py:30: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  tables[table_name] = pd.read_csv(file_path)
  0%|          | 0/100 [00:00<?, ?it/s]

Debug - Available tables: dict_keys(['drugbank-drug_trans_links', 'drugbank-enzymes_polypeptides_ext_id', 'drugbank-drug_manufacturers', 'drugbank-drug_sequences', 'drugbank-drug_external_links', 'drugbank-drug_international_brands', 'drugbank-drug_mixtures', 'drugbank-drug_packagers', 'drugbank-drug_reactions_enzymes', 'drugbank-drug_enzymes_articles', 'drugbank-targets', 'drugbank-drug_carriers_textbooks', 'drugbank-drug_trans_textbooks', 'drugbank-carriers_polypeptides_ext_id', 'drugbank-drugs_attachments', 'drugbank-drug_carriers_articles', 'drugbank-enzymes_polypeptides', 'drugbank-drug_trans_articles', 'drugbank-drug_experimental_properties', 'drugbank-transporters_polypeptides_ext_id', 'drugbank-drug_dosages', 'drugbank-drug_syn', 'drugbank-drug_targ_textbooks', 'drugbank-drug_enzymes_attachments', 'drugbank-drug_carriers_links', 'drugbank-drug_salts', 'drugbank-drugs_articles', 'drugbank-targets_actions', 'drugbank-drug_patents', 'drugbank-drug_affected_organisms', 'drugbank-qu

  1%|          | 1/100 [00:11<19:04, 11.57s/it]

1. question: What is the clinical role of Flumazenil, and how does it differ in terms of administration compared to Lepirudin?
   answer: Flumazenil is used to reverse the effects of benzodiazepine sedation and is typically administered intravenously due to its low oral bioavailability. In contrast, Lepirudin, another drug listed in the "drugbank-drug" table, is a liquid form that Bayer ceased producing in 2012. While the passage discusses Flumazenil's role and preferred administration route, the table provides details about Lepirudin's state and production history.
   text: Target-8693922
   table: drugbank-drug

2. question: How does the type of Flumazenil as an imidazobenzodiazepine compare to the type of Cetuximab listed in the DrugBank database?
   answer: Flumazenil is an imidazobenzodiazepine, which is a benzodiazepine antagonist used to reverse the effects of benzodiazepines. Cetuximab, on the other hand, is categorized as a biotech drug in the DrugBank database, specifically a

...

1. question: What is the significance of bioretrosynthesis in the engineering of biosynthetic pathways, specifically for the antiretroviral drug didanosine, and is it associated with any existing FDA-approved drugs in the database?
   answer: Bioretrosynthesis is significant as it allows for the construction of biosynthetic pathways using the retrograde evolution hypothesis, as demonstrated in the engineered pathway for the antiretroviral nucleoside analog didanosine. Although didanosine's engineered pathway is an application of bioretrosynthesis, it does not appear directly related to the FDA-approved drugs listed in the database, which include leuprolide acetate products and other biotech drugs like Lepirudin, Cetuximab, Dornase alfa, Denileukin diftitox, and Etanercept.
   text: Target-24657930
   table: drugbank-drugs_links

2. question: How does the engineering process described in the passage for didanosine synthesis relate to the biotechnology drugs listed in the database?
   an

 99%|█████████▉| 99/100 [08:54<00:08,  8.11s/it]

1. **Question**: How does glyburide affect the mitoKATP channel under different experimental conditions, and is it included in the DrugBank dataset?
   **Answer**: Glyburide is completely insensitive to the mitoKATP channel when the channel's open state is induced by the absence of ATP and Mg2+, but becomes highly sensitive when the open state is induced by Mg2+, ATP, a physiological opener like GTP, or a pharmacological opener like diazoxide. However, there is no specific entry for glyburide in the provided DrugBank dataset sample.
   **Text**: Target-9593694
   **Table**: drugbank-drug

2. **Question**: Are there any FDA-approved products related to the drugs discussed in the passage, specifically focusing on the effects of glyburide and 5-HD on mitochondria?
   **Answer**: The passage discusses the effects of glyburide and 5-HD on the mitoKATP channel, but the provided tables do not list any FDA-approved products specifically related to these drugs. The tables mainly list other drug

100%|██████████| 100/100 [09:03<00:00,  5.43s/it]

1. question: How is the clinical efficacy of tamoxifen potentially influenced, and which specific genes are investigated for their role in tamoxifen metabolism?
   answer: The clinical efficacy of tamoxifen is potentially influenced by the activity of drug-metabolizing enzymes and transporters involved in its formation, metabolism, and elimination. The study specifically investigates allelic variants of the CYP2D6 enzyme and haplotype-tagging SNPs of transporter genes ABCB1, ABCC2, and ABCG2 for their role in tamoxifen metabolism.
   text: Target-20124171
   table: drugbank-transporters

2. question: Which transporters are relevant to the study of tamoxifen's clinical outcomes, and what organisms are they associated with?
   answer: The transporters relevant to the study of tamoxifen's clinical outcomes are P-glycoprotein 1 (associated with ABCB1), and Canalicular multispecific organic anion transporter 1 (associated with ABCC2), and they are both associated with humans.
   text: Targe

In [11]:
def paraphrase_questions(input_csv, output_csv, model="gpt-4o"):
    """
    Paraphrase questions to remove references to specific tables or passages
    
    Args:
        input_csv: Path to input CSV file with original questions
        output_csv: Path to output CSV file for paraphrased questions
        model: LLM model to use for paraphrasing
    """
    # Read the input CSV
    df = pd.read_csv(input_csv, quoting=csv.QUOTE_ALL)
    
    # New list to hold the updated rows
    paraphrased_data = []
    
    # Process each question
    for _, row in tqdm(list(df.iterrows()), desc="Paraphrasing questions"):
        question = row['question']
        
        # Create prompt for the LLM
        prompt = f"""
        Your task is to paraphrase the following question to remove references to specific tables, datasets, or passages.
        If the question is solely about dataset structure or tables (e.g., "Does the dataset provide any links..."), return "REMOVE" to indicate it should be filtered out.
        Otherwise, paraphrase the question to make it more general while preserving its core content.

        Examples:
        - "Does the data set provide any links or references for Flumazenil in the drugbank-drugs_links table?" → "REMOVE"
        - "Which muscarinic receptor antagonist examined in the passage has affinity values closest to that of atropine and is also a biotech drug listed in the drugbank-drug table?" → "Which muscarinic receptor antagonist has affinity values closest to that of atropine and is also a biotech drug?"
        - "What is the significance of identifying Lys 190 as the modification site of HSA in the study, and how does it contrast with any protein-related drugs in the tables?" → "What is the significance of identifying Lys 190 as the modification site of HSA, and how does it contrast with any protein-related drugs?"

        Original question: {question}
        Paraphrased question:
        """
        
        # Call the LLM for paraphrasing
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an expert in reformulating pharmaceutical questions to make them more general and independent of specific data sources."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        
        paraphrased = response.choices[0].message.content.strip()
        
        # Skip questions that should be removed
        if paraphrased == "REMOVE":
            continue
            
        # Add the paraphrased question and original answer/text/table to the new data
        paraphrased_data.append({
            'question': paraphrased,
            'answer': row['answer'],
            'text': row['text'],
            'table': row['table']
        })
    
    # Write the paraphrased data to the output CSV
    with open(output_csv, 'w', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)
        writer.writerow(['question', 'answer', 'text', 'table'])  # header
        for item in paraphrased_data:
            writer.writerow([
                item['question'],
                item['answer'],
                item['text'],
                item['table']
            ])
    
    print(f"Processed {len(df)} questions, kept {len(paraphrased_data)} after paraphrasing and filtering")
    return paraphrased_data

In [ ]:
# paraphrased_data = paraphrase_questions('both_output.csv', 'paraphrased_output.csv')

In [12]:
def verify_qa_uses_both_sources(input_csv, output_csv, model="gpt-4o"):
    """
    Verify that each QA pair in the CSV actually uses information from both the passage and table
    
    Args:
        input_csv: Path to input CSV file with QA pairs
        output_csv: Path to output CSV file for verified QA pairs
        model: LLM model to use for verification
    """
    # Read the input CSV
    df = pd.read_csv(input_csv, quoting=csv.QUOTE_ALL)
    
    # Load all passages and tables (caching to avoid repeated loading)
    passages = load_target_passages()
    tables = load_drugbank_tables()
    
    # New list to hold the verified rows
    verified_data = []
    
    # Process each QA pair
    for _, row in tqdm(list(df.iterrows()), desc="Verifying QA pairs"):
        question = row['question']
        answer = row['answer']
        passage_id = row['text']
        table_name = row['table']
        
        # Get passage content
        passage_content = passages.get(passage_id, "Passage content not available")
        
        # Get table content (sample)
        table_content = {}
        base_table_name = table_name.replace('.csv', '')
        if base_table_name in tables:
            df_table = tables[base_table_name]
            table_content = {
                'columns': list(df_table.columns),
                'sample': df_table.head(5).to_dict('records')
            }
        else:
            table_content = {"error": f"Table {base_table_name} not found"}
        
        # Create prompt for the LLM
        prompt = f"""
        Your task is to determine if the provided answer to a question uses information from BOTH the passage AND the table.
        
        Question: {question}
        Answer: {answer}
        
        Passage:
        {passage_content[:2000]}  # Limiting to first 2000 chars for token constraints
        
        Table ({table_name}):
        {json.dumps(table_content, indent=2)}
        
        Does the answer draw specific information from BOTH the passage AND the table? Reply with just "YES" or "NO".
        If the answer only uses information from one source or could be answered using only the passage or only the table, reply "NO".
        """
        
        # Call the LLM for verification
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a verification expert who determines if answers draw from multiple data sources."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1
        )
        
        verification = response.choices[0].message.content.strip().upper()
        
        # Only keep pairs that use both sources
        if "YES" in verification:
            verified_data.append({
                'question': question,
                'answer': answer,
                'text': passage_id,
                'table': table_name
            })
    
    # Write the verified data to the output CSV
    with open(output_csv, 'w', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)
        writer.writerow(['question', 'answer', 'text', 'table'])  # header
        for item in verified_data:
            writer.writerow([
                item['question'],
                item['answer'],
                item['text'],
                item['table']
            ])
    
    print(f"Processed {len(df)} QA pairs, kept {len(verified_data)} that use both passage and table")
    return verified_data

# def load_target_passages():
#     """Load all target passages from the pubmed-targets directory"""
#     passages = {}
#     target_files = Path('inputs/pubmed-targets').glob('Target-*')
    
#     for file_path in target_files:
#         target_id = file_path.name
#         with open(file_path, 'r') as f:
#             passages[target_id] = f.read()
#     return passages

# def load_drugbank_tables():
#     """Load all relevant DrugBank tables"""
#     tables = {}
#     csv_files = Path('inputs/drugbank-tables').glob('*.csv')
    
#     for file_path in csv_files:
#         table_name = file_path.stem
#         tables[table_name] = pd.read_csv(file_path)
    
#     return tables

# After running this, you can apply the paraphrasing function to the verified QA pairs
def process_qa_pipeline(input_csv):
    """Complete pipeline to process QA pairs - verify and paraphrase"""
    verified_csv = 'verified_output.csv'
    final_csv = 'paraphrased_output.csv'
    
    # Step 1: Verify QA pairs use both sources
    verify_qa_uses_both_sources(input_csv, verified_csv)
    
    # Step 2: Paraphrase verified questions to remove data source references
    paraphrase_questions(verified_csv, final_csv)
    
    print(f"Process complete. Final output saved to {final_csv}")

In [14]:
process_qa_pipeline('both_output.csv')

/tmp/ipykernel_214670/2028544267.py:30: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  tables[table_name] = pd.read_csv(file_path)
Verifying QA pairs: 100%|██████████| 244/244 [02:31<00:00,  1.61it/s]


Processed 244 QA pairs, kept 77 that use both passage and table


Paraphrasing questions: 100%|██████████| 77/77 [01:49<00:00,  1.42s/it]

Processed 77 questions, kept 77 after paraphrasing and filtering
Process complete. Final output saved to paraphrased_output.csv
